# Evaluation

In [1]:
from pathlib import Path
from evaluation_utils import predict_negations, print_statistics, print_document_text
import pandas as pd
import json
import numpy as np

data_dir = Path('data')
annotation_file = data_dir / 'emc-dcc_ann.json'
dcc_dir = data_dir / 'EMCDutchClinicalCorpus'
result_dir = Path('results')
bilstm_result_file = result_dir / 'bilstm_predictions.csv.gz'

# Load annotated data
with open(annotation_file) as f:
    annotations = json.load(f)

## biLSTM

In [2]:
# Initiate MetaCAT
from medcat.meta_cat import MetaCAT
metacat_model_dir = Path('models') / 'bilstm'
mc_negation = MetaCAT()
mc_negation = mc_negation.load(save_dir=metacat_model_dir, add_prefix_space=True)

# Predict negations
bilstm_predictions = predict_negations(annotations, 'bilstm', mc_negation)

In [3]:
# Save
bilstm_predictions.to_csv(bilstm_result_file, index=False, compression='gzip')

In [4]:
# Load
bilstm_predictions = pd.read_csv(bilstm_result_file)

In [5]:
bilstm_predictions

,entity_id,annotation,bilstm
0,DL1111_32_46,not negated,not negated
1,DL1111_272_280,not negated,not negated
2,DL1111_363_377,not negated,not negated
3,DL1112_22_28,negated,negated
4,DL1113_59_67,not negated,not negated
...,...,...,...
12606,SP2118_862_876,not negated,not negated
12607,SP2119_23_45,negated,negated
12608,SP2120_3_23,not negated,not negated
12609,SP2121_73_85,not negated,not negated


In [6]:
print_statistics(bilstm_predictions, 'bilstm')

tp: 1433
tn: 10722
fp: 116
fn: 340
recall: 0.81
precision: 0.93
specificity: 0.99
accuracy: 0.96
f1: 0.86


## Evaluate examples

In [7]:
bilstm_predictions[bilstm_predictions.entity_id == 'DL2111_108_116']

,entity_id,annotation,bilstm
2791,DL2111_108_116,negated,not negated


In [8]:
# Print a single example of a false negative
print_document_text('DL2111_108_116', dcc_dir)

Geen ochtendstijfheid noch startstijfheid.
In de voorgeschiedenis geen luxatie of subluxatie.
Geen dead arm syndroom.
Geen atrofie.
Beeldvormende diagnostiek: X-linker en rechter schouder 2 richtingen: kleine cyste in het tuberculum majus aan de rechterzijde met bdz.
Echografie: een totale supraspinatuspees ruptuur.
Conclusie: Schouderklachten bij status na totale supraspinatuspees ruptuur.

Entity: syndroom (108-116)


## To do

Output table containing all predictions for easier error analysis. Something like this:

| entity_id | annotation | bilstm | rule_based | bert |
|-|-|-|-|-|
| DL9998_32_46 | not negated | negated | not negated | not negated |
| DL9999_75_87 | negated | not negated | negated | negated |